In [104]:
def center(x,q):
    if q % 1 == 0:
        qhalf = Integer((q-1)/2)
        
        return ((x + qhalf) % q) - qhalf;
    else:
        return 0;

In [105]:
def find_n_cyclotomic_trinomial(lbits,hbits):
    set = [];
    for i in range(1,hbits+1):
        for j in range(0,hbits+1):
            n = 2^i*3^j;
            
            if n > 2^hbits or n < 2^lbits:
                continue;
            else:
                set.append(n);
    
    set.sort();
            
    return set;

In [106]:
def find_ntt_prime_cyclotomic(n,b,lbits,hbits):
    
    fac = factor(n);
    
    if len(fac) > 2 or len(fac) == 0:
        #print("does not support cyclotomic trinomial");
        return 0;

    if len(fac) == 1:
        if fac[0][0] != 2:
            #print("does not support cyclotomic trinomial");
            return 0;
            
    if len(fac) == 2:
        if fac[0][0] != 2 or fac[1][0] != 3:
            #print("does not support cyclotomic trinomial");
            return 0;
        
    if ((n % b) != 0) or (n/2 < b):
        #print("please select adequete b");
        return 0;
    
    w_order = 3*n/b;
    
    qs =[];

    k = 1;

    while True:
        q = w_order*k+1;

        k += 1;

        if q < 2^lbits:
            continue;
            
        if q > 2^hbits:
            break;

        if q in Primes():
            qs.append(q);

    return qs;

In [107]:
def find_generator(q):

    Zq = IntegerModRing(q);

    gs = range(1,q);

    for x in list(factor(q-1)):
        p = x[0];

        t = [];

        for g in gs:
            if Zq(g)^((q-1)/p) != 1:
                t.append(Zq(g));

        gs = t;

    return gs;

In [108]:
def find_w_order_cyclotomic_trinomial(n,b):

    return 3*n/b;

In [109]:
def find_w_cyclotomic_trinomial(n,q,b):

    w_order = find_w_order_cyclotomic_trinomial(n,b);
    
    k = Integer((q-1)/w_order);
    
    ws = [g^k for g in find_generator(q)];
    
    return sorted(list(set(ws)));

In [110]:
def gen_tree(n,q,b):
    
    Zq = IntegerModRing(q);
    
    fac = factor(n/b);
    
    if len(fac) == 2:
        Radix2 = fac[0][1];
        Radix3 = fac[1][1];
    
    if len(fac) == 1:
        if fac[0][0] == 2:
            Radix2 = fac[0][1];
            Radix3 = 0;

        if fac[0][0] == 3:
            Radix2 = 0;
            Radix3 = fac[0][1];

    level = Radix2+Radix3;

    tree = zero_matrix(ZZ,level+1,n/b);
    tree[0,0] = find_w_order_cyclotomic_trinomial(n,b);
    zetas = [];
                
    #Radix-2 NTT with Cyclotomic Polynomial
    tree[1,0] = tree[0,0] / 6;
    tree[1,1] = 5*tree[0,0] / 6;
    
    zetas.append(tree[1,0]);
    
    #Radix-3 NTT
    for l in range(1,Radix3+1):
        for i in range(2*3^(l-1)):
            tree[l+1,3*i  ] = tree[l  ,  i] / 3;
            tree[l+1,3*i+1] = tree[l  ,  i] / 3 + tree[0,0]/3;
            tree[l+1,3*i+2] = tree[l  ,  i] / 3 + 2*tree[0,0]/3;
            
            zetas.append(tree[l+1,3*i]);
            zetas.append((tree[l+1,3*i]*2) % (3*n/b));

    #Radix-2 NTT
    for l in range(Radix3+1,level):
        for i in range(2*3^(Radix3)*2^(l-(Radix3+1))):
            tree[l+1,2*i  ] = tree[l  ,  i] / 2;
            tree[l+1,2*i+1] = tree[l  ,  i] / 2 + tree[0,0]/2;
            
            zetas.append(tree[l+1,2*i]);
    return (zetas, tree);    

In [111]:
def gen_zetas(n,q,d,w):
    
    Zq = IntegerModRing(q);
    
    zetas2 = [center(Integer(Zq(2^16)),q)];

    (zetas, tree) = gen_tree(n,q,b);

    print(tree)
    
    for i in zetas:
        x = Integer(Zq(w)^i * 2^16);
        x = center(x, q);
        zetas2.append(x);
    
    l = len(zetas2)
    str1 = "const int16_t zetas[%d] = {" % l
    print(str1)
    for i in range(ceil(l/8)-1):
        str2 = str(zetas2[8*i])
        print("\t%5d, %5d, %5d, %5d, %5d, %5d, %5d, %5d," %(zetas2[8*i],zetas2[8*i+1],zetas2[8*i+2],zetas2[8*i+3],zetas2[8*i+4],zetas2[8*i+5],zetas2[8*i+6],zetas2[8*i+7]))
        
    if n == 972:
            print("\t%5d, %5d, %5d, %5d" %(zetas2[l-4],zetas2[l-3],zetas2[l-2],zetas2[l-1]))
    else:
        print("\t%5d, %5d, %5d, %5d, %5d, %5d, %5d, %5d" %(zetas2[l-8],zetas2[l-7],zetas2[l-6],zetas2[l-5],zetas2[l-4],zetas2[l-3],zetas2[l-2],zetas2[l-1]))
    
    print("};");
    
    print("====ntt====");
    print("w %d" %center(Integer(Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/3)*2^16),q));

    print("====invntt====");
    print("w %d" %center(Integer(Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/3)*2^16),q));
    z = Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/6);
    print("(z - z^5)^-1 %d" %center(Integer(Zq(2^16) * Zq(z - z^5)^-1),q))
    
    fac = factor(n/b);
    
    if len(fac) == 2:
        Radix2 = fac[0][1];
        Radix3 = fac[1][1];
    
    if len(fac) == 1:
        if fac[0][0] == 2:
            Radix2 = fac[0][1];
            Radix3 = 0;

        if fac[0][0] == 3:
            Radix2 = 0;
            Radix3 = fac[0][1];

    print("level1 %d" %center(Integer(Zq(2^31*2^(-Radix2+1)*3^(-Radix3))),q))
    print("level2 %d" %center(Integer(Zq(2^32*2^(-Radix2+1)*3^(-Radix3))),q))
    print("====reduce.h====");    
    print("QINV : %d" %(q^-1 % 2^16))



In [112]:
n_set = find_n_cyclotomic_trinomial(9,11);
b_set = [1,2,3,4,6];

In [113]:
for n in n_set:
    print("n : ", n);
    for b in b_set:
        qs = find_ntt_prime_cyclotomic(n,b,11,15);
        if qs != 0:
            print("b : ", b, ", qs : ", qs);
    print("");

n :  512
b :  1 , qs :  [7681, 10753, 12289, 15361, 18433, 23041, 26113, 32257]
b :  2 , qs :  [7681, 10753, 12289, 14593, 15361, 18433, 22273, 23041, 26113, 26881, 31489, 32257]
b :  4 , qs :  [2689, 3457, 4993, 6529, 7297, 7681, 9601, 10369, 10753, 12289, 13441, 14593, 15361, 18049, 18433, 20353, 21121, 22273, 23041, 26113, 26497, 26881, 29569, 31489, 31873, 32257]

n :  576
b :  1 , qs :  [3457, 8641, 10369, 12097, 19009]
b :  2 , qs :  [2593, 3457, 8641, 10369, 12097, 16417, 19009, 21601, 25057, 28513, 30241]
b :  3 , qs :  [3457, 6337, 7489, 8641, 10369, 12097, 13249, 14401, 18433, 19009, 20161, 21313, 23041, 26497, 27073, 30529, 32257]
b :  4 , qs :  [2161, 2593, 3457, 3889, 6481, 8209, 8641, 10369, 12097, 15121, 16417, 17713, 19009, 19441, 21169, 21601, 23761, 25057, 28081, 28513, 30241, 32401]
b :  6 , qs :  [2593, 3169, 3457, 6337, 7489, 8353, 8641, 8929, 10369, 10657, 12097, 13249, 13537, 14401, 16417, 16993, 17569, 18433, 19009, 20161, 21313, 21601, 23041, 24481, 25057, 2563

In [114]:
n = 576
q = 3457
b = 4
w = 81

In [115]:
find_w_order_cyclotomic_trinomial(n,b)

432

In [107]:
n = 768
q = 3457
b = 4
w = 22

In [95]:
n = 864
q = 3457
b = 3
w = 9

In [93]:
n = 1152
q = 3457
b = 4
w = 9

In [116]:
gen_zetas(n,q,b,w)

[432   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 72 360   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [108]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [109]:
ws

[2,
 22,
 25,
 27,
 32,
 78,
 88,
 108,
 111,
 124,
 128,
 174,
 190,
 218,
 230,
 251,
 274,
 275,
 294,
 303,
 315,
 341,
 354,
 361,
 400,
 410,
 415,
 417,
 496,
 529,
 562,
 582,
 631,
 673,
 714,
 732,
 742,
 755,
 765,
 779,
 790,
 795,
 858,
 872,
 892,
 910,
 920,
 923,
 940,
 968,
 975,
 1004,
 1022,
 1053,
 1063,
 1082,
 1095,
 1096,
 1100,
 1108,
 1129,
 1176,
 1188,
 1221,
 1248,
 1260,
 1278,
 1282,
 1295,
 1310,
 1350,
 1367,
 1379,
 1391,
 1398,
 1401,
 1408,
 1409,
 1416,
 1427,
 1444,
 1446,
 1458,
 1501,
 1531,
 1535,
 1543,
 1550,
 1588,
 1600,
 1626,
 1668,
 1681,
 1685,
 1709,
 1728,
 1729,
 1748,
 1772,
 1776,
 1789,
 1831,
 1857,
 1869,
 1907,
 1914,
 1922,
 1926,
 1956,
 1999,
 2011,
 2013,
 2030,
 2041,
 2048,
 2049,
 2056,
 2059,
 2066,
 2078,
 2090,
 2107,
 2147,
 2162,
 2175,
 2179,
 2197,
 2209,
 2236,
 2269,
 2281,
 2328,
 2349,
 2357,
 2361,
 2362,
 2375,
 2394,
 2404,
 2435,
 2453,
 2482,
 2489,
 2517,
 2534,
 2537,
 2547,
 2565,
 2585,
 2599,
 2662,
 2

In [60]:
w = 81

In [76]:
n = 768
q = 3457
b = 4
w = 2

In [62]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [63]:
ws

[2,
 22,
 25,
 27,
 32,
 78,
 88,
 108,
 111,
 124,
 128,
 174,
 190,
 218,
 230,
 251,
 274,
 275,
 294,
 303,
 315,
 341,
 354,
 361,
 400,
 410,
 415,
 417,
 496,
 529,
 562,
 582,
 631,
 673,
 714,
 732,
 742,
 755,
 765,
 779,
 790,
 795,
 858,
 872,
 892,
 910,
 920,
 923,
 940,
 968,
 975,
 1004,
 1022,
 1053,
 1063,
 1082,
 1095,
 1096,
 1100,
 1108,
 1129,
 1176,
 1188,
 1221,
 1248,
 1260,
 1278,
 1282,
 1295,
 1310,
 1350,
 1367,
 1379,
 1391,
 1398,
 1401,
 1408,
 1409,
 1416,
 1427,
 1444,
 1446,
 1458,
 1501,
 1531,
 1535,
 1543,
 1550,
 1588,
 1600,
 1626,
 1668,
 1681,
 1685,
 1709,
 1728,
 1729,
 1748,
 1772,
 1776,
 1789,
 1831,
 1857,
 1869,
 1907,
 1914,
 1922,
 1926,
 1956,
 1999,
 2011,
 2013,
 2030,
 2041,
 2048,
 2049,
 2056,
 2059,
 2066,
 2078,
 2090,
 2107,
 2147,
 2162,
 2175,
 2179,
 2197,
 2209,
 2236,
 2269,
 2281,
 2328,
 2349,
 2357,
 2361,
 2362,
 2375,
 2394,
 2404,
 2435,
 2453,
 2482,
 2489,
 2517,
 2534,
 2537,
 2547,
 2565,
 2585,
 2599,
 2662,
 2

In [68]:
n = 864
q = 3457
b = 3
w = 9

In [69]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [70]:
ws

[9,
 24,
 26,
 43,
 66,
 67,
 71,
 98,
 101,
 105,
 133,
 139,
 144,
 148,
 163,
 169,
 179,
 232,
 238,
 244,
 255,
 258,
 280,
 293,
 300,
 309,
 311,
 323,
 324,
 325,
 333,
 351,
 365,
 372,
 384,
 389,
 396,
 402,
 403,
 407,
 416,
 426,
 428,
 447,
 466,
 472,
 482,
 509,
 522,
 542,
 549,
 553,
 557,
 578,
 593,
 623,
 628,
 637,
 638,
 644,
 669,
 671,
 680,
 683,
 688,
 690,
 746,
 753,
 770,
 776,
 798,
 824,
 825,
 841,
 844,
 849,
 855,
 883,
 889,
 891,
 917,
 936,
 962,
 981,
 987,
 1014,
 1019,
 1023,
 1046,
 1056,
 1060,
 1061,
 1062,
 1072,
 1074,
 1083,
 1089,
 1123,
 1136,
 1142,
 1144,
 1148,
 1153,
 1163,
 1177,
 1191,
 1192,
 1193,
 1225,
 1230,
 1231,
 1233,
 1298,
 1323,
 1329,
 1343,
 1412,
 1438,
 1459,
 1487,
 1493,
 1508,
 1513,
 1519,
 1523,
 1547,
 1564,
 1565,
 1568,
 1586,
 1587,
 1589,
 1607,
 1616,
 1654,
 1657,
 1670,
 1677,
 1680,
 1686,
 1699,
 1711,
 1714,
 1727,
 1730,
 1743,
 1746,
 1758,
 1771,
 1777,
 1780,
 1787,
 1800,
 1803,
 1841,
 1850,
 1

In [67]:
w = 9

In [74]:
n = 1152
q = 3457
b = 4
w = 9

In [39]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [40]:
ws

[9,
 24,
 26,
 43,
 66,
 67,
 71,
 98,
 101,
 105,
 133,
 139,
 144,
 148,
 163,
 169,
 179,
 232,
 238,
 244,
 255,
 258,
 280,
 293,
 300,
 309,
 311,
 323,
 324,
 325,
 333,
 351,
 365,
 372,
 384,
 389,
 396,
 402,
 403,
 407,
 416,
 426,
 428,
 447,
 466,
 472,
 482,
 509,
 522,
 542,
 549,
 553,
 557,
 578,
 593,
 623,
 628,
 637,
 638,
 644,
 669,
 671,
 680,
 683,
 688,
 690,
 746,
 753,
 770,
 776,
 798,
 824,
 825,
 841,
 844,
 849,
 855,
 883,
 889,
 891,
 917,
 936,
 962,
 981,
 987,
 1014,
 1019,
 1023,
 1046,
 1056,
 1060,
 1061,
 1062,
 1072,
 1074,
 1083,
 1089,
 1123,
 1136,
 1142,
 1144,
 1148,
 1153,
 1163,
 1177,
 1191,
 1192,
 1193,
 1225,
 1230,
 1231,
 1233,
 1298,
 1323,
 1329,
 1343,
 1412,
 1438,
 1459,
 1487,
 1493,
 1508,
 1513,
 1519,
 1523,
 1547,
 1564,
 1565,
 1568,
 1586,
 1587,
 1589,
 1607,
 1616,
 1654,
 1657,
 1670,
 1677,
 1680,
 1686,
 1699,
 1711,
 1714,
 1727,
 1730,
 1743,
 1746,
 1758,
 1771,
 1777,
 1780,
 1787,
 1800,
 1803,
 1841,
 1850,
 1

In [ ]:
w = 9

In [23]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [24]:
w = min(ws)

In [77]:
gen_zetas(n,q,b,w)

const int16_t zetas[192] = {
	 -147,   886,   460,  1265, -1510,  -460,   867,  1124,
	  257,   723,   722,    -1,   395,  -357, -1521,  1716,
	-1346,  1164,   222,  1611, -1590,  1262,  1484,  -256,
	 1105,  1058,  -603, -1713,   893,  -387,   348,   937,
	  121,  -757,  -820,   216,    44,    39, -1241,  -550,
	   95,  1577,   699,   541,  -502,   655,  -455,  -639,
	 -588, -1464, -1015,   436, -1673,  1267,   281, -1558,
	 1081,  1351,  1331, -1413,  -834,   675,  -205,    54,
	 -641,  -961,    87,  -630,  -548,  -800,   371,   -64,
	 1580, -1428,  -565,   992,   380,  -606,  -661, -1293,
	  176,   156, -1507,  1257,   830,   -50,  -625,    -4,
	-1617,  -569,  1530,  1199,  1449,  -837,  1637,   901,
	 -294,  -732,  1221,   218,   892, -1095, -1588,  -779,
	   22, -1709,  1108,  -275,   968,   858,   354,  1728,
	-1531,  1550, -1367,   124, -1681,  -940, -1379, -1458,
	 1408,  1248, -1685,  -315,  -274,  -400, -1543,   -32,
	-1188, -1053, -1063,  1022,  -417, -1391,  1626,    27,
	 

In [73]:
gen_zetas(n,q,b,w)

const int16_t zetas[192] = {
	 -147,   886,   460,  1265, -1510,  -460,   867,  1124,
	  257,   723,   722,    -1,   395,  -357, -1521,  1716,
	-1346,  1164,   222,  1611, -1590,  1262,  1484,  -256,
	 1105,  1058,  -603, -1713,   893,  -387,   348,   937,
	  121,  -757,  -820,   216,    44,    39, -1241,  -550,
	   95,  1577,   699,   541,  -502,   655,  -455,  -639,
	 -588, -1464, -1015,   436, -1673,  1267,   281, -1558,
	 1081,  1351,  1331, -1413,  -834,   675,  -205,    54,
	 -641,  -961,    87,  -630,  -548,  -800,   371,   -64,
	 1580, -1428,  -565,   992,   380,  -606,  -661, -1293,
	  176,   156, -1507,  1257,   830,   -50,  -625,    -4,
	-1617,  -569,  1530,  1199,  1449,  -837,  1637,   901,
	 -294,  -732,  1221,   218,   892, -1095, -1588,  -779,
	   22, -1709,  1108,  -275,   968,   858,   354,  1728,
	-1531,  1550, -1367,   124, -1681,  -940, -1379, -1458,
	 1408,  1248, -1685,  -315,  -274,  -400, -1543,   -32,
	-1188, -1053, -1063,  1022,  -417, -1391,  1626,    27,
	 

In [16]:
2^16%q

3310

In [85]:
867^-1 % q

1886

In [86]:
(1886 * -66) % q

3433

In [87]:
(1886 * -132) % q

3409

In [79]:
2^15 % q

1655